In [1]:
from model import Model
import pandas as pd
import torch
from transformers import AutoTokenizer

In [2]:
model_sent = Model().to("cuda")
load = torch.load("/home/musasina/projects/teknofest/msnet/model.pth")
model_sent.load_state_dict(load["model_state_dict"])

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


<All keys matched successfully>

In [3]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [4]:
def preprocess(examples):
    model_inputs = tokenizer([examples], max_length=128,padding="max_length",truncation=True,return_tensors="pt")
    return model_inputs["input_ids"][0]

In [5]:
real_results = {0:2,1:1,2:3}

In [6]:
results = []

with open("/home/musasina/projects/teknofest/msnet/datasets/dataset.txt","r") as file:

    for row in file:
        row = row.strip().strip("\n")
        tokenized = preprocess(row)
        label = model_sent(tokenized.to("cuda").view(1,-1)).argmax(dim=1).cpu().item()
        results.append({"text":row,"label":real_results[label]})

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [7]:
pd.DataFrame(results).to_csv("/home/musasina/projects/teknofest/msnet/datasets/emre_data.csv",index=False)